<a href="https://colab.research.google.com/github/y79tian/Machine-learning-Coursera/blob/master/test1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import pyarrow.parquet as pq
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import precision_score, recall_score, accuracy_score, confusion_matrix, roc_curve, auc, classification_report
from sklearn.preprocessing import MinMaxScaler
import sys
import time
import math
import matplotlib.pyplot as plt
from utils import *

class XGBoostClassifier:
  '''Wrapper class for XGBoost Classifer'''

  def __init__(self, fileName='data.pq', aggInterval=['1H'], predictionInterval='1_H', 
               testRatio=0.2, maxDepth=9, learningRate=0.15, subsample=0.9, colsample_bytree=0.3, objective='binary:logistic', numIters=100, earlyStoppingRounds=8):
    '''Initalize XGBoost Model given parameters
      Parameters:
        - fileName (str): name of file to read in data from  (optional: default = 'data.pq')
        - aggInterval ([str]): list of aggregation time intervals (ex: ['1H'] = 1 hour, ['1D'] = 1 day, ['5H'] = 5 hours, ['1T'] = 1 minute, ['1T', '1H] = 1 minute and 1 hour)  (optional: default = ['1H'], 1 hour) (can pass [] for no aggregation)
            for more info about time interval options: https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects
            NOTE: limit 2 time intervals for now
        - predictionInterval (str): prediction time offset (ex: predict craving in '3_H' = 3 hours, '1_H' = 1 day)  (optional: default = '1_H', 1 hour) (can pass '' for no offset, predict craving at current time)
            NOTE: prediction interval must be given in either hours (H), minutes (M), or days (D)
        - testRatio (float): ratio for test data in train-test split (ex: 0.2 = 20%, 80% train)  (optional: default = 0.2)
        - maxDepth (int): maximum depth of tree  (optional: default = 9)
        - learningRate (float): learning rate of XGB model, range [0, 1]  (optional: default = 0.15)
        - subsample (float): fraction of observations to be randomly sampled for each tree (optional: default = 0.9)
        - colsample_bytree (float): fraction of columns to be randomly sampled for each tree (optional: default = 0.3)
        - objective (str): loss function  (optional: default='binary:logistic')
            for more info about objective options: https://xgboost.readthedocs.io/en/latest/parameter.html#learning-task-parameters
        - numIters (int): number of iterations for XGB model  (optional: default = 100)
        - earlyStoppingRounds (int): # of early stopping rounds (optional: default = 8)
    '''
    self.fileName = fileName
    self.aggInterval = aggInterval
    self.predictionInterval = predictionInterval
    self.testRatio = testRatio
    self.maxDepth = maxDepth
    self.learningRate = learningRate
    self.subsample = subsample
    self.colSampleByTree = colsample_bytree
    self.objective = objective
    self.numIters = numIters  
    self.earlyStoppingRounds = earlyStoppingRounds

    if len(self.aggInterval) > 2:
      raise Exception("Number of aggregation intervals is limited to 2")

  def run(self):
    '''Run XGBoost model, including:
      - reading data
      - sampling data
      - cleaning data
      - aggregating by time interval
      - splitting data into training/testing sets
      - training
      - predicting
      - creating log
    '''
    startTime = time.perf_counter()   # keep track of model run time

    # Get data
    print("=====Reading Raw Data=====")
    dataRaw = readDataToDataFrame(self.fileName)  # get data from file into pd.DataFrame
    
    # Sample data (manual resampling for now)
    # For no sampling, use sampleData = dataRaw
    # For sampling, use sampleData= dataRaw.iloc[]  <- sample rows/cols
    sampleData = dataRaw  # example for sampling: dataRaw.iloc[827200:900000,:]  rows 827000-90000, all columns
    
    # Clean data
    # if not using pre-cleaned data, need to clean; otherwise, dataCleaned = sampleData
    # print("=====Cleaning Data=====")
    # dataCleaned = cleanData(sampleData)  # clean data to get it into appropriate format for aggregation/training
    dataCleaned = sampleData
 
    # Perform aggregation
    # if using pre-aggregated data, no need to aggregated (dataAgg = dataCleaned)
    # if need to aggregate, uncomment below
    dataAgg = dataCleaned
    # if self.aggInterval:
    #   print("=====Aggregating Data=====")
    #   dataAgg = aggByTimeInterval(dataCleaned, self.aggInterval)  # aggregate data based on time interval

    # Add prediction interval labels
    # if using pre-labelled data, no need to add prediction labels (predData = dataAgg)
    # if need to add prediction labels, uncomment appropriate line below either 
    # for craving prediction or onset of craving prediction
    predData = dataAgg
    # if self.predictionInterval:
    #   print("=====Adding Interval Predictions=====")
        # NOTE: prediction interval '#_type' type=[H, D, M]
    #   predData = addPredictionIntervalLabels(dataAgg, self.predictionInterval)  
    #   predData = addOnsetPredictionIntervalLabels(dataAgg, self.predictionInterval) 

    # Format final model data
    modelData = predData.iloc[:,:-1]    # remove labels column from training features
    modelLabels = predData.iloc[:,-1]   # labels column
    print("=====Model Ready=====")

    # Split data into training/testing sets
    X_train, X_test, Y_train, Y_test = train_test_split(modelData, modelLabels, 
                                                        test_size=self.testRatio)  # training size: (1 - testRatio), test size: testRatio
    print("=====Finished train-test split=====")
  
    # Normalize features
    X_train, X_test = normalize(X_train, X_test)
    normalized = True

    # Create and fit model
    print("=====Training=====")
    model = xgb.XGBClassifier(max_depth=self.maxDepth, subsample=self.subsample, 
                              colsample_bytree=self.colSampleByTree, objective=self.objective, n_estimators=self.numIters, learning_rate=self.learningRate)
    evalSet = [(X_train, Y_train), (X_test, Y_test)]
    model.fit(X_train, Y_train.values.ravel(), early_stopping_rounds=self.earlyStoppingRounds, eval_metric=["error", "logloss"], eval_set=evalSet, verbose=True)
    
    # Predict with test data
    print("=====Predicting=====")
    y_pred = model.predict(X_test)
    best_preds = [round(value) for value in y_pred]

    endTime =  time.perf_counter()     
    totalRunTime = endTime - startTime   # calculate total model run time

    # Evaluate predictions
    precision = precision_score(Y_test, best_preds, average='macro')
    accuracy = accuracy_score(Y_test, best_preds)
    recall = recall_score(Y_test, best_preds, average='macro')

    print("Precision = {}".format(precision))
    print("Accuracy = {}".format(accuracy))
    print("Recall = {}".format(recall))

    # Confusion Matrix
    cm = confusion_matrix(Y_test, best_preds)
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(cm)
    plt.title('Confusion Matrix')
    fig.colorbar(cax)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()

    # Plot loss/error
    results = model.evals_result()
    epochs = len(results['validation_0']['error'])
    x_axis = range(0, epochs)

    # Log loss
    fig, ax = plt.subplots()
    ax.plot(x_axis, results['validation_0']['logloss'], label='Train')
    ax.plot(x_axis, results['validation_1']['logloss'], label='Test')
    ax.legend()
    plt.ylabel('Log Loss')
    plt.title('XGBoost Log Loss')
    plt.show()

    # Classification Error
    fig, ax = plt.subplots()
    ax.plot(x_axis, results['validation_0']['error'], label='Train')
    ax.plot(x_axis, results['validation_1']['error'], label='Test')
    ax.legend()
    plt.ylabel('Classification Error')
    plt.title('XGBoost Classification Error')
    plt.show()

    # # ROC Curve and AUC
    fpr, tpr, _ = roc_curve(Y_test, best_preds)
    roc_auc = auc(fpr, tpr)
    plt.figure()
    lw = 2
    plt.plot(fpr, tpr, color='darkorange', lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([-0.02, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC curve')
    plt.legend(loc="lower right")
    plt.show()

    # Classification Report
    print(classification_report(Y_test, best_preds))